In [1]:
import numpy as np
import h5py

def get_reshape(dataset):
    """Ein metode som tek inn eit datasett og gjer alle reshapings-tinga for x og y, u og v pluss ribbelista.

    Args:
        dataset (string): filnamnet på hdf5-fila.

    Returns:
        tuple: Ein tuple med x, y, U, V (2d-arrays) og ribs (original-lista)
    """
 
    x,y,Umx,Vmx,I,J,ribs = get_xyuvijribs(dataset)
    
    x_reshape = x.reshape((J,I))
    y_reshape = y.reshape((J,I))
    
    Umx_reshape = Umx.reshape((len(Umx),J,I))
    Vmx_reshape = Vmx.reshape((len(Vmx),J,I))
    
    return x_reshape, y_reshape, Umx_reshape, Vmx_reshape, ribs

def get_xyuvijribs(dataset):
    """Ein metode som hentar ut alle dataa frå ei hdf5-fil og returnerer dei som ndarrays.

    Args:
        dataset (string): filnamnet på hdf5-fila

    Returns:
        tuple: x,y,U,V (alle som 1D-array), I,J (ints) og ribbelista
    """
    with h5py.File(dataset, 'r') as f:
        Umx = np.asarray(f['Umx'])
        Vmx = np.asarray(f['Vmx'])
        (I,J) = f.attrs['I'], f.attrs['J']
        x = np.asarray(f['x'])
        y = np.asarray(f['y'])
        ribs = np.asarray(f['ribs'])
    
    return x,y,Umx,Vmx,I,J,ribs
        
def get_mean(dataset):
    _,_,Umx,Vmx,I,J,_ = get_xyuvijribs(dataset)
    
    u_bar = np.nanmean(Umx,0).reshape((J,I))
    v_bar = np.nanmean(Vmx,0).reshape((J,I))
    
    
    return u_bar, v_bar
    

In [2]:
# x,y,u,v,ribs=get_reshape("data/rib50_Q40_1.hdf5")
# x[0,:]

In [3]:
from datagenerering import tre_objekt
import pickle
import numpy as np
import multiprocessing as mp


In [4]:
with open('data/rib75_Q20_1.pickle', 'rb') as f:
    shared_object = pickle.load(f)


In [5]:
tre = shared_object.tre
ein_t,ein_u = tre[0]

In [6]:
from scipy.spatial import Delaunay

type(ein_t)

scipy.spatial._qhull.Delaunay

In [7]:
def funk(inn, shared_object):
    t,u = shared_object.get_tri_og_U(inn)
    print(inn, t.find_simplex([inn,0,0]))
    return inn + int(t.find_simplex([inn,0,0]))

def funk2(inn):
    
    return shared_object.get_tri_og_U(inn)[0].find_simplex([inn,0,0])

In [8]:

funk(1,shared_object)

manager = mp.Manager()
shared_object = manager.Value(tre_objekt, shared_object)



1 14


In [8]:
input_data = [1, 2, 3, 4, 5]
with mp.Pool(4) as p:
    # Apply the function to the input data in parallel
    results = p.map(funk2, input_data)

In [9]:
results

[array(14, dtype=int32),
 array(108, dtype=int32),
 array(15, dtype=int32),
 array(2896, dtype=int32),
 array(3, dtype=int32)]